# transmission_lines and trans_params

In [1]:
'''
Import based on imports found in the notebooks located here:
Schivley Greg, PowerGenome, (2022), GitHub repository, 
    https://github.com/PowerGenome/PowerGenome/tree/master/notebooks
'''

###
# module issue
import os
import sys
module_path = os.path.abspath(os.getcwd() + '\\..')
if module_path not in sys.path:
    sys.path.append(module_path)
###

from pathlib import Path

import pandas as pd
from powergenome.generators import load_ipm_shapefile
from powergenome.GenX import (
    network_line_loss,
    network_max_reinforcement,
    network_reinforcement_cost,
    add_cap_res_network
)
from powergenome.transmission import (
    agg_transmission_constraints,
    transmission_line_distance,
)
from powergenome.util import (
    init_pudl_connection, 
    load_settings,
    check_settings
)
from statistics import mean

In [2]:
'''
Import based on imports found in the notebooks located here:
Schivley Greg, PowerGenome, (2022), GitHub repository, 
    https://github.com/PowerGenome/PowerGenome/tree/master/notebooks
'''

pudl_engine, pudl_out, pg_engine = init_pudl_connection()
cwd = Path.cwd()

settings_path = (
    cwd / "settings_TD.yml" 
)
settings = load_settings(settings_path)
settings["input_folder"] = settings_path.parent / settings["input_folder"]
# check_settings(settings, pg_engine)

# Information from PowerGenome transmission notebook:
 - PowerGenome compiles information on the transmission constraints between model regions and the centroid-to-centroid distance between model regions.
 - these are used to calculate transmission_lines

In [3]:
'''
pulling in information from PowerGenome transmission notebook
Schivley Greg, PowerGenome, (2022), GitHub repository, 
    https://github.com/PowerGenome/PowerGenome/blob/master/notebooks/Transmission.ipynb
'''
 
transmission = agg_transmission_constraints(pg_engine=pg_engine, settings=settings)
model_regions_gdf = load_ipm_shapefile(settings)

transmission_line_distance(
    trans_constraints_df=transmission,
    ipm_shapefile=model_regions_gdf,
    settings=settings,
)

line_loss = network_line_loss(transmission=transmission, settings=settings)
network_reinforcement_cost = network_reinforcement_cost(transmission=transmission, settings=settings)
network_max_reinforcement = network_max_reinforcement(transmission=transmission, settings=settings)
transmission = agg_transmission_constraints(pg_engine=pg_engine, settings=settings)
add_cap = add_cap_res_network(transmission, settings)

C:\Users\dawso\miniconda3\envs\powergenome\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\dawso\miniconda3\envs\powergenome\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


# Output for SWITCH: transmission_lines

In [4]:
# pulled from SWITCH load_zones file
# need zone_dbid information to populate transmission_line column
def load_zones_table(IPM_regions, zone_ccs_distance_km):
    load_zones = pd.DataFrame(columns=['LOAD_ZONE', 'zone_ccs_distance_km', 'zone_dbid'])
    load_zones['LOAD_ZONE'] = IPM_regions
    load_zones['zone_ccs_distance_km'] = 0 # set to default 0
    load_zones['zone_dbid'] = range(1, len(IPM_regions)+1)
    return load_zones

IPM_regions = settings.get('model_regions')
load_zones = load_zones_table(IPM_regions, zone_ccs_distance_km=0)
zone_dict = dict(zip(load_zones['LOAD_ZONE'].to_list(), load_zones['zone_dbid'].to_list()))

In [5]:
tx_capex_mw_mile_dict = settings.get('transmission_investment_cost')['tx']['capex_mw_mile']
def region_avg(tx_capex_mw_mile_dict, region1, region2):
    r1_value = tx_capex_mw_mile_dict[region1]
    r2_value = tx_capex_mw_mile_dict[region2]
    r_avg = mean([r1_value, r2_value])
    return r_avg
def create_transm_line_col(lz1, lz2, zone_dict):
    t_line = zone_dict[lz1]+'-'+zone_dict[lz2]
    return t_line    

In [6]:
def transmission_lines_table(line_loss, add_cap, tx_capex_mw_mile_dict, zone_dict):
    '''
    Create transmission_lines table based on REAM Scenario 178
    Output Columns:
        TRANSMISSION_LINE: zone_dbid-zone_dbid for trans_lz1 and lz2
        trans_lz1: split PG transmission_path_name
        trans_lz2: split PG transmission_path_name
        trans_length_km: PG distance_mile * need to convert to km (*1.60934)
        trans_efficiency: PG line_loss_percentage (1 - line_loss_percentage)
        existing_trans_cap: PG line_max_cap_flow. Take absolute value and take max of the two values
        trans_dbid: id number
        trans_derating_factor: assuming PG DerateCapRes_1 (0.95)
        trans_terrain_multiplier: 
            trans_capital_cost_per_mw_km * trans_terrain_multiplier = the average of the two regions 
            ('transmission_investment_cost')['tx']['capex_mw_mile'])
        trans_new_build_allowed: how to determine what is allowed. Assume all 1s to start
    '''
    transmission_df = line_loss[['Network_Lines', 'transmission_path_name', 'distance_mile', 'Line_Loss_Percentage']]

    # split to get trans_lz1 and trans_lz2
    split_path_name = transmission_df['transmission_path_name'].str.split('_to_', expand=True)
    transmission_df = transmission_df.join(split_path_name)

    # convert miles to km for trans_length_km
    transmission_df['trans_length_km'] = transmission_df['distance_mile'].apply(
        lambda x: x* 1.609)

    # for trans_efficiency do 1 - line_loss_percentage
    transmission_df['trans_efficiency'] = transmission_df['Line_Loss_Percentage'].apply(
        lambda x: 1-x)

    transmission_df = transmission_df.join(add_cap[['Line_Max_Flow_MW', 'Line_Min_Flow_MW', 'DerateCapRes_1']])

    # want the max value so take abosolute of line_min_flow_mw (has negatives) and then take max
    transmission_df['line_min_abs'] = transmission_df['Line_Min_Flow_MW'].abs()
    transmission_df['existing_trans_cap'] = transmission_df[["Line_Max_Flow_MW", "line_min_abs"]].max(axis=1)

    # get rid of columns
    transm_final = transmission_df.drop(['transmission_path_name', 'distance_mile', 'Line_Loss_Percentage',
                                        'Line_Max_Flow_MW', 'Line_Min_Flow_MW', 'line_min_abs'], axis=1)

    transm_final = transm_final.rename(columns={'Network_Lines':'trans_dbid', 0:'trans_lz1', 1:'trans_lz2', 
                                 'DerateCapRes_1':'trans_derating_factor'})
    
    transm_final['tz1_dbid'] = transm_final['trans_lz1'].apply(lambda x: zone_dict[x])
    transm_final['tz2_dbid'] = transm_final['trans_lz2'].apply(lambda x: zone_dict[x])
    transm_final['TRANSMISSION_LINE'] = transm_final['tz1_dbid'].astype(str)+'-'+transm_final['tz2_dbid'].astype(str)
    # trans_capital_cost_per_mw_km * trans_terrain_multiplier = average of trans_lz1 and trans_lz2
    trans_capital_cost_per_mw_km = min(settings.get('transmission_investment_cost')['tx']['capex_mw_mile'].values()
                                  ) * 1.60934
    transm_final['region_avgs'] = transm_final.apply(lambda row: region_avg(
                                tx_capex_mw_mile_dict, row.trans_lz1, row.trans_lz2), axis=1)
    transm_final['trans_terrain_multiplier'] = transm_final['region_avgs'].apply(lambda x: x / 
                                                                                 trans_capital_cost_per_mw_km)
    
    # set as 1 for now
    transm_final['trans_new_build_allowed'] = 1
    # sort columns
    transm_final = transm_final[['TRANSMISSION_LINE', 'trans_lz1', 'trans_lz2', 'trans_length_km',
                                'trans_efficiency','existing_trans_cap','trans_dbid',
                                'trans_derating_factor','trans_terrain_multiplier','trans_new_build_allowed']]
    return transm_final

In [7]:
transmission_lines = transmission_lines_table(line_loss, add_cap, tx_capex_mw_mile_dict, zone_dict)
transmission_lines

,TRANSMISSION_LINE,trans_lz1,trans_lz2,trans_length_km,trans_efficiency,existing_trans_cap,trans_dbid,trans_derating_factor,trans_terrain_multiplier,trans_new_build_allowed
0,1-2,WEC_BANC,WEC_CALN,109.839257,0.993173,2750.0,1,0.95,1.398089,1
1,2-11,WEC_CALN,WECC_NNV,409.772072,0.974533,100.0,2,0.95,1.009731,1
2,2-12,WEC_CALN,WECC_PNW,750.553013,0.953353,4200.0,3,0.95,1.009731,1
3,2-13,WEC_CALN,WECC_SCE,519.569944,0.967709,3675.0,4,0.95,1.398089,1
4,3-5,WEC_LADW,WECC_AZ,647.181233,0.959777,692.0,5,0.95,1.009731,1
5,3-12,WEC_LADW,WECC_PNW,1052.296348,0.934599,2858.0,6,0.95,1.009731,1
6,3-13,WEC_LADW,WECC_SCE,136.769062,0.991500,3750.0,7,0.95,1.398089,1
7,3-14,WEC_LADW,WECC_SNV,314.807829,0.980435,3883.0,8,0.95,1.009731,1
8,3-15,WEC_LADW,WECC_UT,710.722781,0.955828,1920.0,9,0.95,1.009731,1
9,4-5,WEC_SDGE,WECC_AZ,489.415387,0.969583,1655.0,10,0.95,1.009731,1


# SWITCH: trans_params
* trans_capital_cost_per_mw_km: take min of capex_mw_mile values for regions
* trans_lifetime_yrs: 20 (assuming REAM value)
* trans_fixed_om_fraction: 0.03 (assuming REAM value)

In [8]:
trans_capital_cost_per_mw_km = min(settings.get('transmission_investment_cost')['tx']['capex_mw_mile'].values()
                                  ) * 1.60934
trans_params_table = pd.DataFrame({'trans_capital_cost_per_mw_km':trans_capital_cost_per_mw_km,
                                   'trans_lifetime_yrs':20, 'trans_fixed_om_fraction':0.03},index=[0])
trans_params_table

,trans_capital_cost_per_mw_km,trans_lifetime_yrs,trans_fixed_om_fraction
0,2172.609,20,0.03


In [9]:
transmission_lines.to_csv(r'SWITCH_Inputs\transmission_lines.csv', index = False)
trans_params_table.to_csv(r'SWITCH_Inputs\trans_params.csv', index = False)